Using the ndvi data collection notebook, we can get an image for the "Hauts de France" region and then using the "ville fleuri" dataset, we can cut the image into each city and then calculate the NDVI for each city.


In [51]:
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon

In [ ]:
aoi_coords_wgs84 = [0.527344, 49.174522, 4.394531, 51.144894]

gdf = gpd.GeoDataFrame(geometry=[Point(aoi_coords_wgs84[0], aoi_coords_wgs84[1]), Point(
    aoi_coords_wgs84[2], aoi_coords_wgs84[3])], crs="EPSG:4326")

plate_caree = gdf.to_crs("EPSG:32633")  # UTM zone 33N (Plate Carré equivalent)
aoi_coords_32633 = [plate_caree.geometry[0].x, plate_caree.geometry[0].y,
                    plate_caree.geometry[1].x, plate_caree.geometry[1].y]
aoi_coords_32633

In [ ]:

city_gdf = gpd.read_file("../src/data/villes_villages_fleuris.shp")
# transform to WGS84
city_gdf = city_gdf.to_crs(epsg=32633)
city_gdf.head()

In [ ]:
ndvi_image = "../src/data/ndvi_summer_2022.png"
plt_ndvi_image = plt.imread(ndvi_image)

crs_height = aoi_coords_32633[3] - aoi_coords_32633[1]
crs_width = aoi_coords_32633[2] - aoi_coords_32633[0]

height, width, _ = plt_ndvi_image.shape

print(height, width)
plt.imshow(plt_ndvi_image)


for _, city in city_gdf.iterrows():
    mean_x = city.geometry.centroid.x
    mean_y = city.geometry.centroid.y
    x = (mean_x - aoi_coords_32633[0]) / crs_width * width
    y = (aoi_coords_32633[3] - mean_y) / crs_height * height

    # cut the image to the city geometry
    city_polygon = city.geometry
    xmin, ymin, xmax, ymax = city_polygon.bounds

    # convert the bounds to image coordinates
    xmin_img = int((xmin - aoi_coords_32633[0]) / crs_width * width)
    ymin_img = int((aoi_coords_32633[3] - ymax) / crs_height * height)
    xmax_img = int((xmax - aoi_coords_32633[0]) / crs_width * width)
    ymax_img = int((aoi_coords_32633[3] - ymin) / crs_height * height)

    # Ensure the coordinates are within the image bounds
    xmin_img = max(0, xmin_img)
    ymin_img = max(0, ymin_img)
    xmax_img = min(width, xmax_img)
    ymax_img = min(height, ymax_img)
    
    print("x", xmin_img, xmax_img, "y", ymin_img, ymax_img))

    if xmin_img < xmax_img and ymin_img < ymax_img:
        print(xmin_img, ymin_img, xmax_img, ymax_img)
        cropped_image = plt_ndvi_image[ymin_img:ymax_img, xmin_img:xmax_img, :]
        plt.imshow(cropped_image)
        plt.show()
    else:
        print("Invalid crop dimensions:",
              "x", xmin_img, xmax_img, "y", ymin_img, ymax_img)